In [3]:
import sys
from pathlib import Path

def add_src_to_sys():
    src_path = str((Path() / "../src").resolve())
    if src_path not in sys.path:
        sys.path.append(src_path)

add_src_to_sys()

In [7]:
import os
import random

from collections import defaultdict
from math import ceil

import distinctipy
from PIL import Image, ImageDraw, ImageFont

from plotting.dbg_plots_utils import *
from plotting.dbg_plots import *


random.seed(234)


REFERENCE = "CCAACACAACCACCCACACAAAAACCAAAACA"
READ_POS = [(21,28), (22, 30), (17, 26), (18,27), (7,14), (13,20), (16, 25), (3,11), (0, 10), (6, 13)]
READS = [REFERENCE[start:end] for start, end in READ_POS]

#READS = []
KMER_SIZE = 20
# Open the image
IMAGE_HEIGHT = 600
IMAGE_WIDTH = 800
TEXT_HEIGHT = 14
RECT_MARGIN = 5
TEXT_WIDTH = 9.5
TEXT_SIZE = 14
ERROR_POS = -3
SWAP_COMPLEX = True

footer = True and KMER_SIZE == 3
with_reads = True if READS else False
with_error = True
body = not with_reads

IMAGE_HEIGHT = ceil((len(REFERENCE) - 2) * TEXT_HEIGHT + 4)
IMAGE_WIDTH = ceil((len(REFERENCE) + KMER_SIZE + 1) * TEXT_WIDTH)

kmer_stats, max_kmer_cnt = get_kmer_stats(REFERENCE, KMER_SIZE)
max_mult = max_kmer_cnt

if with_reads:
    # reads_colors = get_reads_colors(READS)
    reads_stats = reads_with_errors_stats(READS, position=ERROR_POS, swap_complex=SWAP_COMPLEX)
    kmer_stats_reads, max_kmer_read_cnt = get_kmer_stats_reads(reads_stats, KMER_SIZE, with_error=with_error, ref=REFERENCE)
    max_mult = max_kmer_read_cnt

HEADER_HEIGHT = TEXT_HEIGHT + len(READS) * TEXT_HEIGHT if with_reads else 0
BODY_HEIGHT = (len(REFERENCE) - KMER_SIZE + 2) * TEXT_HEIGHT if body else 0
FOOTER_HEIGHT = (max_mult + 1) * TEXT_HEIGHT if footer else TEXT_HEIGHT
IMAGE_HEIGHT = HEADER_HEIGHT + FOOTER_HEIGHT + BODY_HEIGHT

image_size = (IMAGE_WIDTH, IMAGE_HEIGHT)
img = Image.new(mode="RGB", size=image_size, color="white")

# Create an ImageDraw object
draw = ImageDraw.Draw(img)
draw.rectangle(((0, 0), (IMAGE_WIDTH - 1, IMAGE_HEIGHT-1)), outline="blue")

# Define the text, position, color, and font
text = REFERENCE
position = (0, 0)

color = "black"  # white
font = ImageFont.load_default(size=TEXT_SIZE)
draw = add_reference_header(draw, text, font, text_color=color, fill_color="lightblue")

if with_reads:
    for row, (t, p) in enumerate(zip(READS, READ_POS)):
        pos = (p[0] * TEXT_WIDTH, (row + 1) * TEXT_HEIGHT)
        fill_color = reads_stats[t]["color"]
        if with_error:
            add_kmer(draw, t, font, text_color=color, fill_color=fill_color, position=pos)
            err_pos = reads_stats[t]["position"]
            err_read = reads_stats[t]["error"]
            add_kmer(draw, err_read[err_pos], font, text_color=color, fill_color="yellow", position=(pos[0] + err_pos * TEXT_WIDTH, pos[1]))
           
        else:
            add_kmer(draw, t, font, text_color=color, fill_color=fill_color, position=pos)

if body:
    for idx in range(0, len(REFERENCE) - KMER_SIZE + 1):
        position = (1 + TEXT_WIDTH * idx, HEADER_HEIGHT + TEXT_SIZE * (idx + 1))
        kmer = REFERENCE[idx : idx + KMER_SIZE]
        #fill_color = (255, 0 , 0)
        add_kmer(draw, kmer, font, text_color=color, fill_color=kmer_stats[kmer]["color"], position=position)
        
if footer:
    if with_reads:
        kmer_positions = get_multiplicity(kmer_stats_reads, image_height=IMAGE_HEIGHT, image_width=IMAGE_WIDTH, text_width=TEXT_WIDTH, text_height=TEXT_HEIGHT, interleave=False)
        for kmer, positions in kmer_positions.items():
            for idx, position in enumerate(positions):
                if kmer_stats_reads[kmer]["errors"][idx]:
                    add_kmer(draw, kmer, font, text_color=color, fill_color=kmer_stats_reads[kmer]["colors"][idx], position=position)
                    error_pos = kmer_stats_reads[kmer]["error_pos"][idx]
                    add_kmer(draw, kmer[error_pos], font, text_color=color, fill_color="yellow", position=(position[0] + error_pos * TEXT_WIDTH, position[1]))
                else:
                    add_kmer(draw, kmer, font, text_color=color, fill_color=kmer_stats_reads[kmer]["colors"][idx], position=position)
    else:
        kmer_positions = get_multiplicity(kmer_stats, image_height=IMAGE_HEIGHT, image_width=IMAGE_WIDTH, text_width=TEXT_WIDTH, text_height=TEXT_HEIGHT)
        
        for kmer, positions in kmer_positions.items():
            for position in positions:
                add_kmer(draw, kmer, font, text_color=color, fill_color=kmer_stats[kmer]["color"], position=position)

# Save the image
wr = "_reads_" if with_reads else ""
wb = "_kmers_" if body else ""
wm = "_mult_" if not with_reads else "_cov_"
wm = wm if footer else ""
we = "_err_" if with_error else ""
wc = "_compl_" if SWAP_COMPLEX else ""
suffix = f"{wr}{wb}{wm}{we}{wc}_K{KMER_SIZE}"
img.save(f"./dbg{suffix}.png")
#img.save(f'./colored_text_boxes_with_text_K{KMER_SIZE}.png')
#img.show()
sequences = READS if not with_error else [read["error"] for read in reads_with_errors_stats(READS, position=ERROR_POS, swap_complex=SWAP_COMPLEX).values()]
sequences = [REFERENCE] if not (with_reads or with_error) else sequences
#sequences.extend(READS)
de_bruijn_graph = generate_de_bruijn_graph(sequences=sequences, k=KMER_SIZE+1)
output_dot_format(de_bruijn_graph, f'de_bruijn_graph{suffix}.dot', kmer_stats)
os.system(f"dot -Tpng de_bruijn_graph{suffix}.dot -o de_bruijn_graph{suffix}.png")

0